## Group Members:
* Veronika Kermoshchuk
* Maria Coughlin
* Sara Conway
* Caitlin Larkin

### Rule-Based Penguins in Pandas
The objective of this task is to categorize or forecast a particular type of penguins using two characteristics accessible in the dataset. Similar to the approach we demonstrated in class on Monday, you will create your own rule-oriented classifier utilizing two threshold parameters for the selected feature.

In [1]:
!pip install pandas
!pip install plotly
!pip install packaging
!pip install ipywidgets

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user install

In [2]:
import pandas as pd
import plotly as plt
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [3]:
plt.offline.init_notebook_mode (connected = True)

In [4]:
# install libraries if needed
# !pip3 install pandas --user codio
# !pip3 install plotly --user codio
# !pip install packaging

In [5]:
df = pd.read_csv("penguins.csv")
df.shape

(344, 7)

In [6]:
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female


### Problem 1
Develop a Python function that receives three arguments, including two threshold values utilized by the rule-oriented classifier and the species type, such as Adelie penguin, you wish to forecast using the classifier. The function should return a Pandas dataframe as the confusion matrix output and precision and recall metrics for your model.

In [7]:
fig = px.scatter(df, x='flipper_length_mm', y='bill_length_mm', color = "species")
fig.show()

In [8]:
df1 = df.copy()
def rule_based_class(flipper_length, bill_length, species):
    predicted_species = "NA"

    if species == 'Adelie':
        if bill_length < 45 and flipper_length <= 205:
            predicted_species = 'Adelie'
        else:
            predicted_species = 'Other'

        df1['species'] = df1['species'].replace(['Chinstrap'], 'Other')
        df1['species'] = df1['species'].replace(['Gentoo'], 'Other')

    elif species == 'Chinstrap':
        if bill_length > 45 and flipper_length < 206:
            predicted_species = 'Chinstrap'
        else:
            predicted_species = 'Other'

        df1['species'] = df1['species'].replace(['Adelie'], 'Other')
        df1['species'] = df1['species'].replace(['Gentoo'], 'Other')

    elif species == 'Gentoo':
        if flipper_length >= 206:
            predicted_species = 'Gentoo'
        else:
            predicted_species = 'Other'

        df1['species'] = df1['species'].replace(['Chinstrap'], 'Other')
        df1['species'] = df1['species'].replace(['Adelie'], 'Other')

    else:
        predicted_species = 'Unknown'
    
    return predicted_species

In [9]:
df['predicted'] = df.apply(lambda x: rule_based_class(x.flipper_length_mm, x.bill_length_mm, "Chinstrap"), axis=1)
actual = df.species.value_counts().to_frame()
conf_matrix = pd.crosstab(df1.species, df["predicted"])
conf_matrix

predicted,Chinstrap,Other
species,,
Chinstrap,56,12
Other,4,272


In [10]:
target = conf_matrix.columns[0]
correctly_predicted = conf_matrix.loc[target, target]
all_predicted = conf_matrix.loc[:,target].sum()
all_actual = conf_matrix.loc[target,:].sum()

In [11]:
print("The precision for", target, "is", round(correctly_predicted/all_predicted,2))

The precision for Chinstrap is 0.93


In [12]:
print("The recall for", target, "is", round(correctly_predicted/all_actual,2))

The recall for Chinstrap is 0.82


### Problem 2
Subsequently, employing the ipwidgets interactive library, enable users to construct their models by designating specific species and thresholds. Your "interact" function must generate a plot, confusion matrix, and precision/recall metrics. Ensure that the output of the model is clearly displayed by correctly color-coding the predicted and actual values and accurately labeling the results.

In [13]:
species = list(df.species.unique())
name1 = ['flipper_length_mm']
name2 = ['bill_length_mm']
minBill = round(df['bill_length_mm'].min(), 1)
maxBill = round(df['bill_length_mm'].max(), 1)
minFlipper = round(df['flipper_length_mm'].min(), 1)
maxFlipper = round(df['flipper_length_mm'].max(), 1)

In [14]:
@interact(type1 = name1, type2 = name2, specie_type = species)
def scatter_by_features(type1, type2, specie_type):
    df['predicted'] = df.apply(lambda x: rule_based_class(x.flipper_length_mm, x.bill_length_mm, specie_type), axis=1)
    selected_df = df[df.species == specie_type]
    fig = px.scatter(selected_df, x = type1, y = type2, color = selected_df['predicted'])
    fig.show()

interactive(children=(Dropdown(description='type1', options=('flipper_length_mm',), value='flipper_length_mm')…